## Calculate ERPs

In [1]:
#Importing files and modules

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
from os import path
%matplotlib notebook
from brainpipe.system import study
from brainpipe.visual import *
from brainpipe.statistics import *
from mne.baseline import rescale
from mne.filter import filter_data
import pandas as pd
from pandas import ExcelWriter



## User variables

In [2]:
low_pass_filter = 10.
sf = 512.
norm_mode = 'mean' #'ratio' 'mean' 'percent' 
baseline = [640 , 768] #-250ms à 0ms
data_to_use = [768, 1536]
n_perm = 200

# Compute ERPs By Odor ALL & selected trials

In [4]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_ByOdor_th40_30_250/')
path2save = path.join (st.path, 'feature/ERP_By_Odor_bipo_250ms_rescale_filtered/')
subjects =  ['LEFC','CHAF', 'VACJ', 'SEMC', 'PIRJ','MICP']
nelec_odors = {
    'CHAF' : [[36, 95], ['1','2','3','4','5','7','8','9']],
    'VACJ' : [[3,61], ['14','15','16','17','10','11','12','13']], 
    'SEMC' : [[0,6,25], ['5','7','8','9','10','11','12','13']],
    'PIRJ' : [[96], ['1','4','9','18','6','7',]],
    'LEFC' : [[0,6,88,91], ['1','2','3','4','14','15','16','17']],
    'MICP' : [[88], ['3','9','14','18','2','6','8','12']],
            }

# ========================== DATAFRAME STRUCTURE ==============================================
writer = ExcelWriter(path2save+'ERP_features_By_Odor.xls')
for su in subjects:
    ix = [ ]
    for elec in nelec_odors[su][0]:
        for odor in nelec_odors[su][1]:
            i=str(elec)+'_'+str(odor) #index unique pour chaque couple odeur/electrode
            ix.append(i)    
    n = len(ix)
    #print (ix, n)
    headers =  ['Subject', 'Elec','Channel','Label','Odor','Max_1st','Max_2nd', 'Max_last', 'Min_1st', 'Min_2nd', 'Min_last',
                        'Max_1st_2nd', 'Max_1st_last', 'Min_1st_2nd', 'Min_1st_last',]
    df = pd.DataFrame(columns = headers, index=ix)
    for h in headers:
        if h in ['Subject','Elec','Channel','Label','Odor' ]:
            df[h] = pd.Series(dtype = 'S', index=ix)
        else:
            df[h] = pd.Series(dtype = 'i', index=ix)
    df['Subject'] = pd.Series([str(su)]*n, index = ix)
            
# ========================== SELECT AND PROCESS ERP DATA ============================================
    for elec in nelec_odors[su][0]:
        for odor in nelec_odors[su][1]:
            data_all = np.load(path.join(path_data, su+'_concat_odor_'+odor+'_bipo.npz'))
            data, channel, label = data_all['x'][elec], data_all['channel'][elec], data_all['label'][elec]
            ntrials = data.shape[1]
            #print ('Odeur : ',odor, '\Elec data : ', data.shape, 'Channel : ', channel, 'Label : ', label, 'N_trials :', ntrials,)

            #Filter data for one elec (all trials):
            data = np.array(data, dtype='float64')
            data_to_filter = np.swapaxes(data, 0, 1)
            filtered_data = filter_data(data_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
            filtered_data = np.swapaxes(filtered_data, 0, 1)
            #print ('Size of filtered data:', filtered_data.shape,)

            #Normalize the non-averaged data (all trials)
            times = np.arange(filtered_data.shape[0])
            #print ('time points : ', times.shape)
            filtered_data_to_norm = np.swapaxes(filtered_data, 0, 1)
            norm_filtered_data = rescale(filtered_data_to_norm, times=times, baseline=baseline, mode=norm_mode)
            norm_filtered_data = np.swapaxes(norm_filtered_data, 0, 1)
            #print ('Size norm & filtered data : ', norm_filtered_data.shape,)

            #Select the time window to compute
            all_trials_data = norm_filtered_data[range(data_to_use[0], data_to_use[1])] #ERP features only after stimulus
            #print('-> Shape of all trials : ', all_trials_data.shape,)
            
            i=str(elec)+'_'+str(odor)
            
            df.loc[i, 'Elec'] = str(elec)
            df.loc[i, 'Channel'] = str(channel)
            df.loc[i, 'Label'] = str(label)
            df.loc[i, 'Odor'] = str(odor)
            df.loc[i, 'Max_1st'] = all_trials_data[:,0].max()
            df.loc[i, 'Max_2nd'] = all_trials_data[:,1].max()
            df.loc[i, 'Max_last'] = all_trials_data[:,-1].max()
            df.loc[i, 'Min_1st'] = all_trials_data[:,0].min()
            df.loc[i, 'Min_2nd']= all_trials_data[:,1].min()
            df.loc[i, 'Min_last'] = all_trials_data[:,-1].min()
            df.loc[i, 'Max_1st_2nd'] = all_trials_data[:,0].max() - all_trials_data[:,1].max()
            df.loc[i, 'Max_1st_last'] = all_trials_data[:,0].max() - all_trials_data[:,-1].max()
            df.loc[i, 'Min_1st_2nd']= all_trials_data[:,0].min() - all_trials_data[:,1].min()
            df.loc[i, 'Min_1st_last'] = all_trials_data[:,0].min() - all_trials_data[:,-1].min()
    #print (df)
    df.to_excel(writer, sheet_name=su)
writer.save()            
    



-> Olfacto loaded
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Applying baseline correction (mode: mean)
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Applying baseline correction (mode: mean)
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Applying baseline correction (mode: mean)
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Applying baseline correction (mode: mean)
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Applying baseline correction (mode: mean)
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Applying baseline corr

In [ ]:


# ======================= COMPUTE ERP FEATURES AND APPEND DATAFRAME =====================================
            #features to append in df
            df.loc[odor, 'Elec'] = str(elec)
            df.loc[odor, 'Max_1st'] = all_trials_data[:,0].max()
            df.loc[odor, 'Max_2nd'] = all_trials_data[:,1].max()
            df.loc[odor, 'Max_last'] = all_trials_data[:,-1].max()
            df.loc[odor, 'Min_1st'] = all_trials_data[:,0].min()
            df.loc[odor, 'Min_2nd']= all_trials_data[:,1].min()
            df.loc[odor, 'Min_last'] = all_trials_data[:,-1].min()
            
            df.loc[odor, 'Max_1st_2nd'] = all_trials_data[:,0].max() - all_trials_data[:,1].max()
            df.loc[odor, 'Max_1st_last'] = all_trials_data[:,0].max() - all_trials_data[:,-1].max()
            df.loc[odor, 'Min_1st_2nd']= all_trials_data[:,0].min() - all_trials_data[:,1].min()
            df.loc[odor, 'Min_1st_last'] = all_trials_data[:,0].min() - all_trials_data[:,-1].min()

#         df.append()
#     df.to_excel(writer, sheet_name=str(su))
#     print (df)
# #     #Save all the plots
# #     rep = path.join(st.path, 'feature/ERP_By_Odor_bipo_250ms_rescale_filtered/',su)
# #     fname = (rep + '_ERP_Odor_'+str(label[elec])+'_'+str(channel[elec])+'_('+str(elec)+').png')
# #     print (fname)
# #     plt.show()
# #     plt.savefig(fname, dpi=300, bbox_inches='tight')
# #     plt.clf()
# #     plt.close
# #     del label, elec, channel
